In [1]:
import os
import numpy as np
import xarray as xr
import dask
import dask.array as da
from dask.distributed import Client, SSHCluster
from dask.diagnostics import Profiler

In [2]:
%autoawait asyncio

async def setup_dask_cluster():
    """the SSHCluster API is asynchronous and must be awaited
    - the scheduler runs on the first host in the list.
    - to run multiple workers on a host, repeat its name in the list."""
    preload_command = "import os; os.umask(0o000)"
    cluster = await SSHCluster(
        hosts=["localhost", "panoseti-dfs0", "panoseti-dfs1", "panoseti-dfs2", "localhost"],
        connect_options={"known_hosts": None},
        worker_options={
            "nthreads": 16,
            "memory_limit": "16GB",
            # "preload": "/mnt/beegfs/worker_init.py",
            "preload": [preload_command]
        },
        scheduler_options={"port": 0, "dashboard_address": ":8797"},
    )

    # Connect to the cluster asynchronously.
    client = await Client(cluster, asynchronous=True)
    
    print(f"Dask dashboard link: {client.dashboard_link}")
    
    # Returning the client and cluster to use them later
    return client, cluster

# here we can run the async function directly -> with autoawait
client, cluster = await setup_dask_cluster()
client

2025-09-07 19:22:36,911 - distributed.deploy.ssh - INFO - 2025-09-07 19:22:36,909 - distributed.scheduler - INFO - State start
2025-09-07 19:22:36,926 - distributed.deploy.ssh - INFO - 2025-09-07 19:22:36,925 - distributed.scheduler - INFO -   Scheduler at:     tcp://10.0.1.22:45867
2025-09-07 19:22:40,690 - distributed.deploy.ssh - INFO - 2025-09-08 02:22:31,994 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.0.1.10:42315'
2025-09-07 19:22:41,380 - distributed.deploy.ssh - INFO - 2025-09-08 02:22:39,287 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.0.1.38:39019'
2025-09-07 19:22:41,657 - distributed.deploy.ssh - INFO - 2025-09-08 03:30:04,466 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.0.1.18:42055'
2025-09-07 19:22:41,948 - distributed.deploy.ssh - INFO - 2025-09-07 19:22:41,947 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.0.1.22:43109'
2025-09-07 19:22:43,489 - distributed.deploy.ssh - INFO - 2025-09-08 02:22:34,79

Dask dashboard link: http://10.0.1.22:8797/status


<Client: 'tcp://10.0.1.22:45867' processes=3 threads=48, memory=44.70 GiB>

In [6]:
from pathlib import Path
import numcodecs
import zarr

l0_dir = Path('/mnt/beegfs/data/L0')
l1_dir = Path('/mnt/beegfs/data/L1')

run_dir = 'obs_Lick.start_2024-07-25T04:34:06Z.runtype_sci-data.pffd'
pff_base = "start_2024-07-25T05:27:22Z.dp_img16.bpp_2.module_1.seqno_2"
# pff_base = "start_2024-07-25T04:34:46Z.dp_img16.bpp_2.module_1.seqno_0"

zarr_in_path = l1_dir / run_dir / f"{pff_base}.zarr"
out_path = l1_dir / run_dir / f'atype_img-medsub.{pff_base}.zarr'
print(zarr_in_path)
print(out_path)

assert os.path.exists(zarr_in_path)
# os.makedirs(out_path, exist_ok=True)

ds = xr.open_datatree(
    zarr_in_path,
    consolidated=False,
    engine='zarr',
    chunks={},
    cache=True
)
ds

/mnt/beegfs/data/L1/obs_Lick.start_2024-07-25T04:34:06Z.runtype_sci-data.pffd/start_2024-07-25T05:27:22Z.dp_img16.bpp_2.module_1.seqno_2.zarr
/mnt/beegfs/data/L1/obs_Lick.start_2024-07-25T04:34:06Z.runtype_sci-data.pffd/atype_img-medsub.start_2024-07-25T05:27:22Z.dp_img16.bpp_2.module_1.seqno_2.zarr


<xarray.DataTree>
Group: /
│   Dimensions:     (time: 1574804, y: 32, x: 32)
│   Dimensions without coordinates: time, y, x
│   Data variables:
│       timestamps  (time) float64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
│       images      (time, y, x) uint16 3GB dask.array<chunksize=(8192, 32, 32), meta=np.ndarray>
└── Group: /headers
    ├── Group: /headers/quabo_0
    │       Dimensions:   (time: 1574804)
    │       Dimensions without coordinates: time
    │       Data variables:
    │           pkt_tai   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           pkt_num   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           tv_usec   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           tv_sec    (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           pkt_nsec  (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    ├── Group: /headers/quabo_2
    │       Dimensions:   (time: 1574804)
    │       Dimensions without coordinates: time
    │       Data variables:
    │           pkt_num   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           tv_sec    (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           tv_usec   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           pkt_tai   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           pkt_nsec  (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    ├── Group: /headers/quabo_3
    │       Dimensions:   (time: 1574804)
    │       Dimensions without coordinates: time
    │       Data variables:
    │           pkt_tai   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           tv_usec   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           pkt_num   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           pkt_nsec  (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    │           tv_sec    (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
    └── Group: /headers/quabo_1
            Dimensions:   (time: 1574804)
            Dimensions without coordinates: time
            Data variables:
                tv_sec    (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
                pkt_num   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
                pkt_tai   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
                tv_usec   (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>
                pkt_nsec  (time) int64 13MB dask.array<chunksize=(16384,), meta=np.ndarray>

In [7]:
# out_path = 'img_medsub.zarr'
# out_path = l1_dir / 'img_medsub.zarr'
original_umask = os.umask(0o000)

try:
    if 'ph' in str(zarr_in_path):
        if ds.images.shape[1] == 32:
            pre_preprocessed = images_swapped
            add_baselines = pre_preprocessed + 800
            pedestal = add_baselines.median(dim=['time'])
            pedestal_sub = add_baselines - pedestal
            
            # define the threshold and the boolean mask lazily
            sigma_5 = pedestal_sub.std('time') * 5
            sigma_5_above_mask = (pedestal_sub > sigma_5)
            
            # create the final masked array lazily
            ph_5_sigma_above = pedestal_sub.where(sigma_5_above_mask)
        else:
            pre_preprocessed = ds.images[:1000]
        
            add_baselines = pre_preprocessed + 800
            pedestal = add_baselines.median(dim=['time'])
            pedestal_sub = add_baselines - pedestal
            
            # define threshold and the boolean mask lazily
            sigma_5 = pedestal_sub.std('time') * 5
            sigma_5_above_mask = (pedestal_sub > sigma_5)
            
            ph_5_sigma_above = pedestal_sub.where(sigma_5_above_mask)
        
        #  Compute only the final result, once.
        #  call executes the entire optimized graph
        store_operation = ph_5_sigma_above.to_zarr(
            out_path,
            mode='w',
            consolidated=False,
            compute=False,
            zarr_format=3,
        )
    
    elif 'img' in str(zarr_in_path):
        frame_step = 200
        
        # convert to int32 to avoid uint16 overflow during subtraction
        img_int32 = ds.images.astype('int32')
        
        # 8x8 block medians on a strided subset, then median over time
        block_8x8_medians = (
            img_int32[::frame_step]
            .coarsen(y=8, x=8, boundary="trim")
            .median()
            .median('time')
        )  # dims: (y, x) with sizes 4x4
        
        # upsample 4x4 medians to a 32x32 Dask array using da.repeat
        upsampled_medians_da = da.repeat(block_8x8_medians.data, 8, axis=0)
        upsampled_medians_da = da.repeat(upsampled_medians_da, 8, axis=1)
    
        # wrap in an xarray.DataArray to restore coordinates for broadcasting
        upsampled_medians = xr.DataArray(
            upsampled_medians_da,
            dims=('y', 'x'),
            coords={'y': ds.images.y, 'x': ds.images.x},
        )
        
        # subtract spatial 8x8 medians via broadcasting (no map_blocks needed)
        median_subtraction_8x8 = img_int32 - upsampled_medians  # (time, y, x)
        
        # cmpute 32x32 medians, then subtract
        # median_subtraction_8x8 = img_adc_to_pe
        supermedian_img = median_subtraction_8x8[::frame_step].median('time')
        median_subtraction_final = median_subtraction_8x8 - supermedian_img
        median_subtraction_final_pe = median_subtraction_final / 1.5
        # median_subtraction_final = median_subtraction_8x8
    
        # define the desired compressor using numcodecs
        # Zstd is an excellent modern compressor balancing speed and ratio.
        # Level 10 is a high compression setting.
        
        compressor = zarr.codecs.ZstdCodec(level=7)
    
        # create the encoding dictionary
        # this tells xarray to use our chosen compressor for the data variable.
        median_subtraction_final.name = 'median_subtracted_data'
        encoding = {
            median_subtraction_final.name: {"compressors": [compressor]}
        }
        
        # call to_zarr with the encoding
        # it's best practice to set consolidated=True for faster read performance
        store_operation = median_subtraction_final.to_zarr(
            out_path,
            mode='w',
            consolidated=True,
            encoding=encoding,
            compute=False,
            zarr_format=3
        )
    result = store_operation.compute()
finally:
    os.umask(original_umask)

/mnt/beegfs/miniconda3/envs/zarr-py313/lib/python3.13/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(
/mnt/beegfs/miniconda3/envs/zarr-py313/lib/python3.13/site-packages/zarr/api/asynchronous.py:233: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [5]:
ds_clean = xr.open_datatree(
    out_path,
    consolidated=False,
    engine='zarr',
    chunks={},
    cache=True
)

ds_clean.images

AttributeError: 'DataTree' object has no attribute 'images'

In [ ]:
cluster.shutdown()
client.shutdown()